# Introduction

La gestion de portefeuille est un domaine crucial de la finance. Il s'agit non seulement de sélectionner les bons investissements, mais aussi de comprendre comment les combiner de manière optimale pour atteindre un équilibre idéal entre le risque et le rendement. Un des modèles les plus célèbres et influents dans ce domaine est la Théorie Moderne du Portefeuille (TMP), introduite par Harry Markowitz en 1952.

Le principe fondamental de la TMP est que les investisseurs cherchent à maximiser leur rendement pour un niveau de risque donné. C'est ici qu'intervient le concept de diversification - en combinant différents actifs dont les rendements ne sont pas parfaitement corrélés, on peut réaliser une performance globale plus stable et donc moins risquée.

La variance du portefeuille, c'est-à-dire la variabilité de ses rendements, est souvent utilisée comme mesure du risque dans la TMP. En minimisant la variance, on cherche à rendre les rendements du portefeuille aussi stables et prévisibles que possible.

Dans ce notebook, nous allons utiliser la TMP pour optimiser un portefeuille d'investissements. Nous utiliserons des techniques d'optimisation pour déterminer la répartition idéale des actifs qui minimisera la variance du portefeuille. Les poids que nous obtiendrons seront utilisés pour améliorer notre stratégie de trading.

Nous allons passer par plusieurs étapes pour réaliser cela :

-  **Collecte des données** : Nous utiliserons des données historiques sur les rendements des actifs dans notre portefeuille.

- **Calcul des rendements et de la covariance** : Nous utiliserons ces données pour estimer les rendements attendus et la covariance des rendements des actifs.

- **Optimisation du portefeuille** : Nous utiliserons un algorithme d'optimisation pour déterminer les poids de chaque actif dans le portefeuille qui minimisera la variance.

En appliquant ces étapes, nous obtiendrons une stratégie d'investissement optimisée qui peut nous aider à prendre des décisions plus informées et, espérons-le, à obtenir de meilleurs rendements.

Commençons donc sans plus tarder !

In [124]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, norm, spearmanr, pearsonr
from scipy.optimize import minimize
import MetaTrader5 as mt5
import sys
sys.path.append(r'C:\Users\ftiag\Desktop\Business, trading et investissement\Business\modules')
from research_tools import get_clean_mt5_data

mt5.initialize()

login_mt5 = 1051534030
mdp_mt5 = 'FG2SF2M74R'
server = 'FTMO-Demo'

mt5.login(login_mt5, mdp_mt5, server)

True

Dans cette partie du code, nous commençons par définir les actifs de notre portefeuille. Ce sont les symboles des paires de devises que nous allons analyser. En utilisant les identifiants de ces paires `('GBPJPY', 'GBPUSD', 'EURGBP', 'EURUSD', 'USDCAD', 'USDJPY')`, nous pouvons obtenir les informations de marché pertinentes pour notre analyse.

Nous utilisons ensuite une fonction appelée `get_clean_mt5_data` pour récupérer les données de ces actifs. Cette fonction fait partie d'un module personnel que j'ai créé pour faciliter l'analyse des données de marché. Elle utilise l'API MetaTrader 5 (MT5) pour récupérer les données de marché, et renvoie une version "nettoyée" de ces données.

Le paramètre `interval` spécifie l'intervalle de temps pour les données que nous récupérons. Ici, nous utilisons mt5.TIMEFRAME_H4, ce qui signifie que nous récupérons des données sur un intervalle de 4 heures.

Le paramètre `n_bars` spécifie le nombre de barres (ou périodes) que nous voulons récupérer. Ici, nous utilisons `5000`, ce qui signifie que nous récupérons les données des 5000 dernières périodes.

Enfin, le `.close` à la fin de l'appel de la fonction signifie que nous nous intéressons uniquement aux prix de clôture pour chaque période.

Voici donc comment nous pouvons lire cette ligne de code :

"Pour chaque paire de devises dans notre liste de tickers, récupérons les 5000 derniers prix de clôture sur des intervalles de 4 heures à l'aide de la fonction `get_clean_mt5_data` de notre module personnel."

In [125]:
tickers = ['GBPJPY', 'GBPUSD', 'EURGBP', 'EURUSD', 'USDCAD', 'USDJPY']
prices = get_clean_mt5_data(tickers, interval=mt5.TIMEFRAME_H4, n_bars=5000).close

Maintenant que nous avons récupéré nos données de prix, nous devons les préparer pour notre analyse.

La première ligne `prices = prices.unstack(0).fillna(method='ffill')` reformate les données de prix en une structure plus appropriée pour notre analyse. L'opération `unstack(0)` réorganise le DataFrame pour que chaque colonne corresponde à une paire de devises. `fillna(method='ffill')` remplit les valeurs manquantes avec la dernière valeur valide connue, ce qui garantit qu'il n'y a pas de trous dans nos données.

Ensuite, nous initialisons nos poids avec une distribution uniforme, c'est-à-dire que nous supposons initialement que chaque actif dans notre portefeuille a le même poids. Nous utilisons l'expression `np.array([1/len(returns.columns)]*len(returns.columns))` pour cela.

Nous définissons ensuite la fonction `minimize_variance`. Cette fonction prend un ensemble de poids et calcule la variance du portefeuille qui en résulte. Les rendements sont calculés en utilisant `prices.pct_change().dropna()`, qui calcule le changement en pourcentage des prix d'une période à l'autre. Ensuite, nous calculons la matrice de covariance des rendements avec `returns.cov()`, qui nous donne une mesure de la façon dont les rendements de différents actifs évoluent ensemble. La variance du portefeuille est calculée en utilisant `np.dot(np.dot(weights.T, cov), weights)`, qui est une formulation standard pour calculer la variance d'un portefeuille à partir des poids des actifs et de leur matrice de covariance.

Nous définissons également une contrainte sur les poids dans la fonction `constraint1`. Cette contrainte garantit que la somme des poids est égale à 1, ce qui signifie que nous utilisons la totalité de notre capital pour investir.

Ensuite, nous utilisons la fonction `minimize` du module d'optimisation `scipy.optimize` pour trouver les poids qui minimisent la variance du portefeuille. La méthode `SLSQP` (Sequential Least Squares Programming) est utilisée comme méthode d'optimisation. Les contraintes et les limites de poids sont également passées à la fonction `minimize`.

La variable `result` contiendra les résultats de l'optimisation, y compris les poids optimaux du portefeuille.

In [126]:
prices = prices.unstack(0).fillna(method='ffill')
index = prices.columns
weights = np.array([1/len(returns.columns)]*len(returns.columns))
def minimize_variance(weights):
    weights.reshape(-1, 1)
    returns = prices.pct_change().dropna()
    cov = returns.cov()
    portfolio_variance = float(np.dot(np.dot(weights.T, cov), weights))
    return portfolio_variance * 10000

def constraint1(x):
    return np.sum(x) - 1

con1 = {'type': 'eq', 'fun': constraint1}
constraints = [con1]
bnds = [(0, None) for _ in range(len(weights))]
result = minimize(minimize_variance, weights, method='SLSQP', bounds=bnds, constraints=constraints)

Dans le premier bloc de code, nous imprimons le résultat de notre optimisation. La sortie nous indique que l'optimisation a réussi. La clé 'fun' dans le dictionnaire de sortie est la valeur minimale de la fonction objectif, c'est-à-dire la variance minimale du portefeuille que nous avons trouvée. Les valeurs sous la clé 'x' sont les poids optimaux pour chaque actif de notre portefeuille qui minimisent la variance.

Ensuite, nous créons un dictionnaire qui relie chaque paire de devises à son poids optimal dans le portefeuille. Nous utilisons la fonction `zip` pour combiner les indices (les noms des paires de devises) et les poids optimaux. Le résultat est un dictionnaire où chaque paire de devises est liée à son poids optimal.

Finalement, nous sauvegardons ce dictionnaire de poids optimaux dans un fichier pickle pour une utilisation future. Le module `pickle` est utilisé pour sérialiser et désérialiser les objets Python. En sérialisant notre dictionnaire de poids optimaux, nous pouvons facilement le charger à partir du fichier pickle à une date ultérieure. Cela peut être particulièrement utile si le processus d'optimisation est coûteux en termes de temps et de ressources, car nous n'aurons pas à réexécuter l'optimisation chaque fois que nous aurons besoin des poids optimaux.

Dans l'ensemble, ce bloc de code représente l'étape finale de notre processus d'optimisation du portefeuille. En minimisant la variance du portefeuille, nous avons créé une stratégie de portefeuille qui, selon la théorie moderne du portefeuille, devrait nous offrir le rendement le plus stable pour un niveau de risque donné.

In [127]:
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.002848667171112722
       x: [ 3.019e-01  0.000e+00  0.000e+00  3.227e-01  2.466e-01
            1.288e-01]
     nit: 15
     jac: [ 6.008e-03  1.128e-02  1.080e-02  5.310e-03  5.937e-03
            5.483e-03]
    nfev: 106
    njev: 15

In [128]:
dict(zip(index, list(result.x)))

{'EURGBP': 0.30189857553076915,
 'EURUSD': 0.0,
 'GBPJPY': 0.0,
 'GBPUSD': 0.32268924221792383,
 'USDCAD': 0.24659302521026463,
 'USDJPY': 0.12881915704104258}

In [131]:
import pickle
with open('weights.pkl', 'wb') as f:
    pickle.dump(dict(zip(index, list(result.x))), f)